#  Count coins on the tray

# Task Description

On the images tray1.jpg to tray8.jpg, find coins of either 5 zł or 5 gr denomination.

1. Using the Hough transform, detect as many coins as possible in the images.
2. Find the edges of the tray.
3. Determine the area of the tray.
4. Count the coins on and off the tray.
5. Calculate the total value of the coins on and off the tray.

# Opis zadania

Na obrazach tray1.jpg . . . tray8.jpg znajdź monety 5 zł lub 5 gr.

1. Stosując transformatę Hougha znajdź możliwie dużo monet na rysunkach
2. Znajdź krawędzie tacy,
3. Określ obszar tacy,
4. Zlicz monety na tacy i poza nią,
5. Określ sumę na tacy i poza nią.

In [1]:
import cv2 as cv
import numpy as np

# Read images
img_lines = cv.imread('tray8.jpg', cv.IMREAD_COLOR)
img_circles = cv.imread('tray8.jpg', 0)

# Detect lines
gray = cv.cvtColor(img_lines, cv.COLOR_BGR2GRAY)
edges = cv.Canny(gray, 50, 150, apertureSize=3)
lines = cv.HoughLinesP(edges, 1, np.pi / 180, 90, minLineLength=100, maxLineGap=10)

# Detect circles
img = cv.medianBlur(gray, 5)
circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, 20, param1=100, param2=50, minRadius=0, maxRadius=0)
circles = np.uint16(np.around(circles))

# Determine tray boundaries
x_max = np.max(lines[:, 0, 0])
x_min = np.min(lines[:, 0, 2])
y_max = np.max(lines[:, 0, 1])
y_min = np.min(lines[:, 0, 3])

# Initialize counters and sums
count_5zl_inside = 0
count_5gr_inside = 0
count_5zl_outside = 0
count_5gr_outside = 0
sum_5zl_inside = 0
sum_5gr_inside = 0
sum_5zl_outside = 0
sum_5gr_outside = 0

# Determine the expected size of a 5 zł coin
five_size = max(circles[0, :, 2]) * 0.90

# Process each detected circle
for circle in circles[0, :]:
    x, y, r = circle
    print("radius: ", r)

    # Check if the center of the circle is inside the tray
    if x_min <= x <= x_max and y_min <= y <= y_max:
        # Check which coin it is
        if r > five_size:
            count_5zl_inside += 1
            sum_5zl_inside += 5
            cv.circle(img_lines, (x, y), 2, (255, 0, 0), 4)
        else:
            count_5gr_inside += 1
            sum_5gr_inside += 0.05
            cv.circle(img_lines, (x, y), 2, (0, 255, 0), 4)

        # Draw a circle to mark the coin as inside the tray
        cv.circle(img_lines, (x, y), r, (255, 20, 147), 2)
    else:
        # Check which coin it is
        if r > five_size:
            count_5zl_outside += 1
            sum_5zl_outside += 5
            cv.circle(img_lines, (x, y), 2, (255, 0, 0), 4)
        else:
            count_5gr_outside += 1
            sum_5gr_outside += 0.05
            cv.circle(img_lines, (x, y), 2, (0, 255, 0), 4)

        # Draw a circle to mark the coin as outside the tray
        cv.circle(img_lines, (x, y), r, (255, 0, 0), 2)

# Print the results
print("Coins inside tray:")
print("5 zł:", count_5zl_inside, "total value:", sum_5zl_inside)
print("5 gr:", count_5gr_inside, "total value:", sum_5gr_inside)
print("Coins outside tray:")
print("5 zł:", count_5zl_outside, "total value:", sum_5zl_outside)
print("5 gr:", count_5gr_outside, "total value:", sum_5gr_outside)

# blend the two images
# img_circles_color = cv.cvtColor(img_circles, cv.COLOR_GRAY2BGR)
# result = cv.addWeighted(img_lines, 0.5, img_circles_color, 0.5, 0)

# Display the resulting image with lines and circles
# cv.imshow('Result', result)
cv.imshow('Lines', img_lines)
# cv.imshow('Circles', img_circles_color)
cv.waitKey(0)
cv.destroyAllWindows()

radius:  36
radius:  29
radius:  31
radius:  31
radius:  30
radius:  30
radius:  30
radius:  37
radius:  29
radius:  29
radius:  29
radius:  30
Coins inside tray:
5 zł: 1 total value: 5
5 gr: 4 total value: 0.2
Coins outside tray:
5 zł: 1 total value: 5
5 gr: 6 total value: 0.3
